Se entrena un modelo SVM con el dataset Iris, se calcula su precisión y guarda el modelo y un archivo con información sobre él.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import json
import os
import joblib
import time
import platform
from pathlib import Path
import sklearn

# Paths
ARTIFACTS_DIR = Path("artifacts")
ARTIFACTS_DIR.mkdir(exist_ok=True)

print("scikit-learn version:", sklearn.__version__)
print("Python version:", platform.python_version())

# Cargar dataset
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)  # Convertir a DataFrame
y = pd.Series(iris.target)

# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Modelo SVM con kernel RBF
model = SVC(kernel='rbf', gamma='scale', C=1.0, probability=True)

# Entrenar
model.fit(X_train, y_train)

# Predecir
y_pred = model.predict(X_test)

# Precisión
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Guardar modelo
model_path = ARTIFACTS_DIR / "model.pkl"
joblib.dump(model, model_path)
print("Artefacto guardado en:", model_path.resolve())

# Crear manifest
manifest = {
    "name": "iris_svm_model",
    "created_at": time.strftime("%Y-%m-%d %H:%M:%S"),
    "framework": "scikit-learn",
    "sklearn_version": sklearn.__version__,
    "python_version": platform.python_version(),
    "features": list(X.columns),
    "target": "iris_target",
    "test_accuracy": float(accuracy)
}

# Guardar manifest
manifest_path = ARTIFACTS_DIR / "model_card.json"
with open(manifest_path, "w", encoding="utf-8") as f:
    json.dump(manifest, f, indent=2, ensure_ascii=False)

print("Manifest guardado en:", manifest_path.resolve())


scikit-learn version: 1.6.1
Python version: 3.12.11
Accuracy: 1.0
Artefacto guardado en: /content/artifacts/model.pkl
Manifest guardado en: /content/artifacts/model_card.json


Verifica que existan los archivos del modelo y del manifest; si faltan, lanza un error. Si existen, carga el modelo entrenado y la información del manifest para poder usarlos.

In [ ]:
if not model_path.exists() or not manifest_path.exists():
    raise FileNotFoundError(
        "No se encontraron los artefactos requeridos. "
        "Ejecuta primero el notebook '01 — Serialización del Modelo (Pipeline + Manifest)' "
        "para generar 'artifacts/knn_pipeline.pkl' y 'artifacts/model_card.json'."
    )

loaded_model = joblib.load(model_path)
with open(manifest_path, "r", encoding="utf-8") as f:
    manifest = json.load(f)

print("Modelo cargado con éxito.")
manifest

Modelo cargado con éxito.


{'name': 'iris_svm_model',
 'created_at': '2025-09-29 09:04:12',
 'framework': 'scikit-learn',
 'sklearn_version': '1.6.1',
 'python_version': '3.12.11',
 'features': ['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'],
 'target': 'iris_target',
 'test_accuracy': 1.0}

In [ ]:
X.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)'],
      dtype='object')

Verifica que el manifest contenga las claves obligatorias (features, target, sklearn_version) y avisa si alguna falta. Luego extrae las características, la variable objetivo y la versión de scikit-learn usada para entrenar. Finalmente, imprime esta información y alerta si la versión actual de scikit-learn es distinta a la usada en el entrenamiento.

In [ ]:
required_manifest_keys = ["features", "target", "sklearn_version"]
missing_keys = [k for k in required_manifest_keys if k not in manifest]
if missing_keys:
    print("ADVERTENCIA: el manifiesto no contiene las claves:", missing_keys)

declared_features = manifest.get("features", [])
declared_target = manifest.get("target", None)
trained_sklearn_version = manifest.get("sklearn_version", None)

print("Features declaradas:", declared_features)
print("Target declarado:", declared_target)
print("Versión sklearn entreno:", trained_sklearn_version, "| actual:", sklearn.__version__)
if trained_sklearn_version and sklearn.__version__ != trained_sklearn_version:
    print("⚠️ ADVERTENCIA: La versión actual de scikit-learn difiere de la utilizada en el entrenamiento.")

Features declaradas: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Target declarado: iris_target
Versión sklearn entreno: 1.6.1 | actual: 1.6.1


Estas funciones permiten validar los datos de entrada y realizar predicciones individuales o en lote usando el modelo SVM entrenado del Iris, incluyendo probabilidades si el modelo las soporta.

In [ ]:
from typing import Tuple
import json, os, joblib, platform, sklearn
import numpy as np, pandas as pd
from pathlib import Path
from typing import Iterable, Dict, Any, List, Optional

# Columnas esperadas según manifiesto (fallback a dos columnas comunes)
REQUIRED_COLUMNS = declared_features if declared_features else ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)','petal width (cm)']

def validate_input(df_in: pd.DataFrame) -> pd.DataFrame:
    """Valida que df_in contenga las columnas requeridas y devuelve X en el orden correcto."""
    if not isinstance(df_in, pd.DataFrame):
        raise TypeError("La entrada debe ser un pandas.DataFrame.")
    missing = [c for c in REQUIRED_COLUMNS if c not in df_in.columns]
    if missing:
        raise ValueError(f"Faltan columnas requeridas: {missing}. Se esperaban: {REQUIRED_COLUMNS}")
    # devolvemos solo las columnas requeridas en el orden correcto
    X = df_in[REQUIRED_COLUMNS].copy()
    return X

def predict_batch(df_in: pd.DataFrame, return_proba: bool = True):
    X = validate_input(df_in)
    yhat = loaded_model.predict(X)
    proba = None
    if return_proba and hasattr(loaded_model, "predict_proba"):
        proba = loaded_model.predict_proba(X)  # matriz (n, n_classes)
    return yhat, proba

def predict_one(record: Dict[str, Any], return_proba: bool = True):
    df_one = pd.DataFrame([record])
    yhat, proba = predict_batch(df_one, return_proba=return_proba)
    out = {"input": record, "prediction": int(yhat[0])}
    if proba is not None:
        probs = proba[0].tolist()
        out["proba"] = probs
        out["p1"] = float(probs[-1])  # probab

Este bloque toma un lote de 4 flores, valida las columnas, predice su especie con el SVM entrenado y devuelve tanto las predicciones como las probabilidades asociadas a cada clase.

In [ ]:
ejemplo_batch = pd.DataFrame({
    "sepal length (cm)": [5.1, 6.2, 4.7, 7.0],
    "sepal width (cm)":  [3.5, 2.9, 3.2, 3.1],
    "petal length (cm)": [1.4, 4.3, 1.3, 5.9],
    "petal width (cm)":  [0.2, 1.3, 0.2, 2.1]
})

yhat, proba = predict_batch(ejemplo_batch, return_proba=True)
print("Predicciones:", yhat)
print("Probabilidades:", proba)



Predicciones: [0 1 0 2]
Probabilidades: [[0.97133834 0.01849568 0.01016598]
 [0.00992297 0.97096493 0.0191121 ]
 [0.97262617 0.01758478 0.00978905]
 [0.00936364 0.0042873  0.98634906]]


Este bloque sirve para probar que la API Flask está activa antes de enviar predicciones u otras peticiones.

In [ ]:
import os, requests, json


BASE = os.getenv("API_BASE", "http://127.0.0.1:5000")
API_KEY = os.getenv("API_KEY")  # si definiste auth, se enviará

headers = {"Content-Type": "application/json"}
if API_KEY:
    headers["X-API-KEY"] = API_KEY

# Health
try:
    print("GET /health ->", requests.get(f"{BASE}/health", headers=headers, timeout=5).json())
except Exception as e:
    print("No se pudo conectar a la API (¿app.py corriendo?):", e)

GET /health -> {'model_path': '/content/artifacts/model.pkl', 'ok': True, 'python': '3.12.11', 'sklearn_loaded': '1.6.1'}


In [ ]:
# Model info
try:
    print("GET /model-info ->", requests.get(f"{BASE}/model-info", headers=headers, timeout=5).json())
except Exception as e:
    print("No se pudo conectar a la API:", e)

GET /model-info -> {'features': ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)'], 'name': 'iris_svm_model', 'sklearn_version_train': '1.6.1', 'target': 'iris_target'}


Este bloque envía 4 flores a la API y recibe de vuelta las predicciones y probabilidades calculadas por el modelo SVM entrenado del Iris.

In [ ]:
payload = [
    {"sepal length (cm)": 5.1, "sepal width (cm)": 3.5, "petal length (cm)": 1.4, "petal width (cm)": 0.2},
    {"sepal length (cm)": 6.2, "sepal width (cm)": 2.8, "petal length (cm)": 4.8, "petal width (cm)": 1.8},
    {"sepal length (cm)": 7.0, "sepal width (cm)": 3.2, "petal length (cm)": 4.7, "petal width (cm)": 1.4},
    {"sepal length (cm)": 5.9, "sepal width (cm)": 3.0, "petal length (cm)": 5.1, "petal width (cm)": 1.8},
]

try:
    r = requests.post(f"{BASE}/predict", headers=headers, data=json.dumps(payload), timeout=5)
    print("POST /predict ->", r.json())
except Exception as e:
    print("No se pudo conectar a la API:", e)


POST /predict -> {'ok': True, 'result': [{'p1': 0.010165982742031905, 'petal length (cm)': 1.4, 'petal width (cm)': 0.2, 'prediction': 0, 'sepal length (cm)': 5.1, 'sepal width (cm)': 3.5}, {'p1': 0.5788543571230381, 'petal length (cm)': 4.8, 'petal width (cm)': 1.8, 'prediction': 2, 'sepal length (cm)': 6.2, 'sepal width (cm)': 2.8}, {'p1': 0.07861622583133035, 'petal length (cm)': 4.7, 'petal width (cm)': 1.4, 'prediction': 1, 'sepal length (cm)': 7.0, 'sepal width (cm)': 3.2}, {'p1': 0.8468102827366086, 'petal length (cm)': 5.1, 'petal width (cm)': 1.8, 'prediction': 2, 'sepal length (cm)': 5.9, 'sepal width (cm)': 3.0}]}
